# Инициализация библиотек

In [1]:
! gdown 13zFsjH4BfggyQuQjL2F4a0EWpIPClU50 -q
! pip install https://github.com/scikit-learn-contrib/scikit-learn-extra/archive/master.zip -q
! pip install xlwt -q
! pip install pyclustering -q
! pip install kmodes

     / 740.1 kB 5.8 MB/s 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.0/100.0 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 30.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import xlwt

from sklearn.preprocessing import OrdinalEncoder
from sklearn.datasets import make_blobs
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.model_selection import train_test_split
from kmodes.kmodes import KModes


import numpy as np
from sklearn.metrics import pairwise_distances
from sklearn.metrics.cluster import contingency_matrix

from sklearn.cluster import KMeans
from sklearn_extra.cluster import KMedoids
from sklearn.metrics import davies_bouldin_score
from sklearn.metrics import precision_score
from sklearn.metrics import silhouette_score
from sklearn.metrics import calinski_harabasz_score
from sklearn.metrics import adjusted_rand_score
from sklearn.metrics import v_measure_score


from sklearn.cluster import Birch
from sklearn.cluster import AgglomerativeClustering

In [3]:
from pyclustering.cluster.rock import rock as rock_cluster
from pyclustering.utils import read_sample
from pyclustering.samples.definitions import SIMPLE_SAMPLES

# Фунции для обработки датасета и подсчета метрик

In [4]:
# Функция для рассчета инерции
def inertia(X, y, n_clusters):
  inertia = 0
  y = y.reshape(-1, 1)
  for i, label in enumerate(range(n_clusters)):
      cluster_center = X[(y == i)[:, 0]].mean(axis=0)
      cluster_inertia = ((X[(y == i)[:, 0]] - cluster_center) ** 2).sum()
      inertia += cluster_inertia

  return inertia

def calculate_cdbw(X, labels, n_clusters):
    labels = labels.reshape(-1, 1)
    k = len(np.unique(labels))
    centroids = np.zeros((n_clusters, X.shape[1]))

    # считаем межклассовое расстояние
    for i in range(n_clusters):
        cluster_indices = np.where(labels == i)[0]
        centroids[i,:] = np.mean(X[cluster_indices,:], axis=0)

    intra_dists = [np.mean(pairwise_distances(X[(labels == i)[:, 0]], [centroids[i]], metric='euclidean')) for i in range(k)]
    inter_dists = [np.min(pairwise_distances(centroids[np.arange(k) != i], [centroids[i]], metric='euclidean')) for i in range(k)]
    return np.mean(intra_dists) / np.mean(inter_dists)



# Функция для рассчета метрик и вывода их на экран
def showing_metrics(df_enc_cutted, labels, true_labels, n_clusters):
  n_clusters = n_clusters
  distances = np.zeros((n_clusters,))

  for i in range(n_clusters):
      cluster_indices = np.where(labels == i)[0]
      cluster_distances = pairwise_distances(df_enc_cutted[cluster_indices,:], metric='euclidean')
      distances[i] = np.mean(cluster_distances)
      
  within_cluster_distance = np.mean(distances)
  print('Within cluster distance:', within_cluster_distance)

  n_clusters = n_clusters
  centroids = np.zeros((n_clusters, df_enc_cutted.shape[1]))

  # считаем межклассовое расстояние
  for i in range(n_clusters):
      cluster_indices = np.where(labels == i)[0]
      centroids[i,:] = np.mean(df_enc_cutted[cluster_indices,:], axis=0)

  db_index = davies_bouldin_score(df_enc_cutted, labels)

  num_clusters = n_clusters
  compactness = 0
  for i in range(num_clusters):
    cluster_indices = (labels == i)
    if sum(cluster_indices) == 0:
        continue
    cluster_center = df_enc_cutted[cluster_indices].mean(axis=0)
    distances = pairwise_distances(df_enc_cutted[cluster_indices], cluster_center.reshape(1, -1))
    compactness += np.sum(distances)
  
  cs_score = calinski_harabasz_score(df_enc_cutted, labels)


  
  between_cluster_distance = pairwise_distances(centroids, metric='euclidean').mean()
  print('Between cluster distance:', between_cluster_distance)
  
  precision = precision_score(true_labels, labels, average='binary')
  print("Precision: ", precision)

  print('Индекс плотности (CDbw):', calculate_cdbw(df_enc_cutted, labels, n_clusters))
  silhouette_avg = silhouette_score(df_enc_cutted, labels)
  print("Silhouette Score: ", silhouette_avg)

  print('Сalinski_harabasz_score:', cs_score)

  print('DB index:', db_index)
  
  print('Индекс компактности', compactness)

  print('Score function: ', ((inertia(df_enc_cutted, labels, 1)
   - inertia(df_enc_cutted, labels, 2)) / inertia(df_enc_cutted, labels, 1)))
  print('RAND index: ', adjusted_rand_score(true_labels.ravel(), labels))
  print('V measure : ', v_measure_score(true_labels.ravel(), labels))

  print('*' * 30)

In [5]:
# Функция для прогона алгоритмов с датасетом и вывода метрик на экран
def main_alg(df, true_labels, n_clusses):
  np.random.seed(123)

  # Алгоритм Kmedoids
  kmedoids = KMedoids(n_clusters=n_clusses)
  kmedoids.fit(df)
  kmedoids_labels = kmedoids.predict(df)
  print('Для алгоритма kmedoids:')
  showing_metrics(df, kmedoids_labels, true_labels, n_clusses)

  # Алгоритм Birch
  brc = Birch(n_clusters=n_clusses)
  brc.fit(df)
  birch_labels = brc.predict(df)
  print('Для алгоритма Birch:')
  showing_metrics(df, birch_labels, true_labels, n_clusses)

  # Алгоритм KMeans
  kmeans = KMeans(n_clusters=n_clusses, random_state=0, n_init="auto").fit(df)
  kmeans_labels = kmeans.labels_
  print('Для алгоритма KMeans:')
  showing_metrics(df, kmeans_labels, true_labels, n_clusses)

  # Алгоритм AgglomerativeClustering 
  agg_clustering = AgglomerativeClustering(n_clusters=n_clusses)
  agl_labels = agg_clustering.fit_predict(df)
  print('Для алгоритма AgglomerativeClustering:')
  showing_metrics(df, agl_labels, true_labels, n_clusses)

  # Алгоритм KModes
  km = KModes(n_clusters=2, init='Huang', n_init=5, verbose=0)
  kmodes_labels = km.fit_predict(df)
  print('Для алгоритма KModes:')
  showing_metrics(df, kmodes_labels, true_labels, n_clusses)

  return kmedoids_labels.reshape(-1, 1), agl_labels.reshape(-1, 1), \
         kmeans_labels.reshape(-1, 1), birch_labels.reshape(-1, 1), \
         kmodes_labels.reshape(-1, 1)


# Для набора данных с грибами грибов

In [6]:
df = pd.read_excel('/content/Грибы.xls')

In [7]:
enc = OrdinalEncoder()
enc.fit(df['Класс'].values.reshape(-1, 1))
# съедобный - 0, ядовитый - 1
y = enc.transform(df['Класс'].values.reshape(-1, 1))

In [8]:
df = df.drop(['Класс'], axis=1)

In [9]:
enc = OrdinalEncoder()
enc.fit(df)
X = enc.transform(df)

In [10]:
# Необязательно инвентирование истинных y
y = [1 if x == 0 else 0 for x in y]
y = np.array(y).reshape(-1, 1)

In [11]:
kmedoids_labels, agl_labels, kmeans_labels, birch_labels, kmodes_labels = main_alg(X, y, 2)

Для алгоритма kmedoids:
Within cluster distance: 11.045442427901252
Between cluster distance: 1.4939965982280405
Precision:  0.5545529122231337
Индекс плотности (CDbw): 2.6695088848450093
Silhouette Score:  0.03719110147570848
Сalinski_harabasz_score: 267.83216331737634
DB index: 5.339017769690001
Индекс компактности 64766.37743475559
Score function:  -1.492710539895362
RAND index:  0.008861761253433515
V measure :  0.005896863809761612
******************************
Для алгоритма Birch:
Within cluster distance: 9.935505252629577
Between cluster distance: 3.7680472011339368
Precision:  0.4859392575928009
Индекс плотности (CDbw): 0.9499904232213522
Silhouette Score:  0.19516153940808273
Сalinski_harabasz_score: 2136.698866477588
DB index: 1.899980846442704
Индекс компактности 58365.122482147985
Score function:  -0.6965361979686737
RAND index:  0.0035270814338467017
V measure :  0.0023223168105042905
******************************
Для алгоритма KMeans:
Within cluster distance: 9.72429460

In [12]:
columns = list(df.columns)
columns = columns + ['Истинные метки', 'k-medoids', 'Agglomerative', 'kmeans_labels', 'Birch', 'KModes']

In [13]:
total = pd.DataFrame(np.concatenate((df.values, y, kmedoids_labels, agl_labels, kmeans_labels, birch_labels, kmodes_labels), axis=1), columns=columns)

In [14]:
total.to_excel('total_data_mashtoms.xls')

<ipython-input-14-e4a78b5f38f3>:1: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  total.to_excel('total_data_mashtoms.xls')


In [27]:
# Добавление очень медленного алгоритма, который нужен по условию задачи
from pyclustering.cluster.clarans import clarans
# Создание экземпляра класса CLARANS
clarans_instance = clarans(X, number_clusters=2, numlocal=2, maxneighbor=3)
# Запуск алгоритма
clarans_instance.process()
# Получение результирующих кластеров
clusters = clarans_instance.get_clusters()
labels = np.array([1 * (x in clusters[0]) for x in range(len(X))])

In [30]:
showing_metrics(X, labels, y, 2)